In [12]:
import tensorflow as tf
import numpy as np

In [53]:
# Load the model
class SequentialNN(tf.keras.models.Sequential):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def predict(self, x):
        pred_probs = super().predict(x)
        rounded_probs = tf.math.round(pred_probs)
        predictions = tf.cast(rounded_probs, tf.int32).numpy()
        return predictions.flatten()
    

model = tf.keras.models.load_model('trustee_model.h5', custom_objects={'SequentialNN': SequentialNN})

# load train and test data
X_train = np.load('train_data.npy')
y_train = np.load('train_labels.npy', allow_pickle=True)
X_test = np.load('test_data.npy')
y_test = np.load('test_labels.npy', allow_pickle=True)

# X_train = X_train.astype('float32')
# y_train = np.array([label[0] for label in y_train]).astype('int32')

# X_test = X_test.astype('float32')
# y_test = np.array([label[0] for label in y_test]).astype('int32')


In [54]:
# attacking the model

X_test_final = np.load('test_data.npy')
y_test_final = np.load('test_labels.npy', allow_pickle=True)


model.evaluate(X_test_final, y_test_final, verbose=2)

1/1 - 0s - loss: 2.2926 - accuracy: 0.9231 - 73ms/epoch - 73ms/step


[2.2925775051116943, 0.9230769276618958]

In [55]:
# change second column of X_test_final to 0
X_test_final[:, 1] = 100

In [56]:
model.evaluate(X_test_final, y_test_final, verbose=2)

1/1 - 0s - loss: 470.6077 - accuracy: 0.5000 - 12ms/epoch - 12ms/step


[470.60772705078125, 0.5]

In [57]:
newX = []
for x in X_train:
    x.flatten()
    newX.append(x.flatten())

newX = np.array(newX)
newX.shape

(101, 30)

In [59]:
newY = []
for y in y_train:
    if y[0] == 1:
        newY.append(0)
    elif y[1] == 1:
        newY.append(1)
    elif y[2] == 1:
        newY.append(2)
    
newY = np.array(newY)
newY.shape

(101,)

In [60]:
from trustee import ClassificationTrustee
from sklearn.metrics import classification_report

clf = model
y_pred = clf.predict(X_test)

trustee = ClassificationTrustee(expert=clf)
trustee.fit(newX, newY, num_iter=50, num_stability_iter=10, samples_size=0.3, verbose=True)
dt, pruned_dt, agreement, reward = trustee.explain()
dt_y_pred = dt.predict(X_test)
dt_y_pred = pruned_dt.predict(X_test)

print("Model explanation global fidelity report:")
print(classification_report(y_pred, dt_y_pred))
print("Model explanation score report:")
print(classification_report(y_test, dt_y_pred))

1/1 [==============================] - 0s 42ms/step
Initializing training dataset using <__main__.SequentialNN object at 0x3170098e0> as expert model


2024-05-01 12:09:46.487450: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at conv_ops_fused_impl.h:761 : INVALID_ARGUMENT: convolution input must be 4-dimensional: [1,32,30]


InvalidArgumentError: Graph execution error:

Detected at node sequential_nn/conv1d/Conv1D defined at (most recent call last):
  File "/Users/nairda/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/Users/nairda/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code

  File "/Users/nairda/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/nairda/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance

  File "/Users/nairda/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start

  File "/Users/nairda/anaconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start

  File "/Users/nairda/anaconda3/lib/python3.9/asyncio/base_events.py", line 601, in run_forever

  File "/Users/nairda/anaconda3/lib/python3.9/asyncio/base_events.py", line 1905, in _run_once

  File "/Users/nairda/anaconda3/lib/python3.9/asyncio/events.py"

In [ ]:
# from sklearn.metrics import roc_curve, roc_curve, auc
# import matplotlib.pyplot as plt

# # Compute macro-average ROC curve and ROC area

# # First aggregate all false positive rates
# # all_fpr = [ np.array([0,0,1]), np.array([0,1,0]), np.array([1,0,0]) ]
# all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# # Then interpolate all ROC curves at this points
# mean_tpr = np.zeros_like(all_fpr)
# for i in range(n_classes):
#     mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# # Finally average it and compute AUC
# mean_tpr /= n_classes

# fpr["macro"] = all_fpr
# tpr["macro"] = mean_tpr
# roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# # Plot all ROC curves
# plt.figure()
# plt.plot(fpr["micro"], tpr["micro"],
#      label='micro-average ROC curve (area = {0:0.2f})'
#            ''.format(roc_auc["micro"]),
#      color='deeppink', linestyle=':', linewidth=4)

# plt.plot(fpr["macro"], tpr["macro"],
#      label='macro-average ROC curve (area = {0:0.2f})'
#            ''.format(roc_auc["macro"]),
#      color='navy', linestyle=':', linewidth=4)

# colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
# for i, color in zip(range(n_classes), colors):
#     plt.plot(fpr[i], tpr[i], color=color, lw=lw,
#          label='ROC curve of class {0} (area = {1:0.2f})'
#          ''.format(i, roc_auc[i]))

# plt.plot([0, 1], [0, 1], 'k--', lw=lw)
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Some extension of Receiver operating characteristic to multi-class')
# plt.legend(loc="lower right")
# plt.show()

In [ ]:
# predictions_no_attack = model.predict(X_test)
# predictions_attack = model.predict(X_test_final)

# from sklearn.metrics import roc_curve, roc_curve, auc
# import matplotlib.pyplot as plt

# fprn, tprn, thresholdsn = roc_curve(y_test, predictions_no_attack)
# fpra, tpra, thresholdsa = roc_curve(y_test_final, predictions_attack)

# roc_auc_c = auc(fprn, tprn)
# roc_auc_a = auc(fpra, tpra)

# fontSize = 14
# labelSize = 17
# # Plot the ROC curve
# plt.figure(figsize=(8, 6))
# plt.plot(fprn, tprn, color='darkorange', lw=2, label=f'No attack (AUC={roc_auc_c:.2f})', \
#         marker='o')
# plt.plot(fpra, tpra, color='red', lw=2, label=f'attack (AUC={roc_auc_a:.2f})', \
#         marker='*')
# plt.plot([0,1], [0,1], color='navy', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.01])
# plt.tick_params(axis='both', which='major', labelsize=labelSize)
# plt.xlabel('False Positive Rate', fontsize=fontSize)
# plt.ylabel('True Positive Rate', fontsize=fontSize)
# plt.legend(loc='lower right', fontsize=fontSize-1)
# plt.grid(True)
# plt.show()